# Оценка рыночной стоимости мотоцикла с пробегом

## Выполнили: Болгарин Максим, Моисеенков Павел. BD-11

<br>

**Цель проекта:** проанализировать факторы, влияющие на рыночную стоимость подержанных мотоциклов, и научиться её предсказывать. Какую пользу можно вынести из этого простому человеку? Зная, как образуется стоимость мотоцикла, владелец подобного транспортного средства может подобрать подходящую для него цену для продажи. С другой стороны, будущий владелец сможет оценить адекватность заинтересовавшего его предложения.

**План выполнения:**
1. Поиск данных, формирование датасета
    * Скачивание и парсинг данных.
    * Формирование датасета.
2. Первичный анализ данных
    * Изучение данных: определение смысла признаков, типов данных, наличия пропусков и т.д.
3. Предобработка данных
    * Очистка и отбор данных, выполнение необходимых преобразований.
4. Визуальный анализ данных
    * Поиск скрытых особенностей в данных, анализ корреляций.
5. Применение моделей анализа данных для предсказания стоимости
    * Тестирование нескольких простых моделей, сравнение качества, выбор baseline.
    * Обучение и проверка модели, показывающей наилучшее качество.
6. Вывод

<br>

### 1. Поиск данных, формирование датасета

Источником данных послужил архив объявлений на сайте moto.drom.ru

Всего было обработано около 7000 объявлений из Москвы. Скачивание производилось в [другом Jupyter Notebook](https://github.com/maxbolgarin/datamining_project/blob/master/project/parce.ipynb). На выходе мы имеем .csv файл, с ним нам и предстоит работать.

#### Подключим необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd

#### Теперь загрузим данные

In [2]:
motorcycles = pd.read_csv('data/motorcycles.csv', index_col='id')

In [3]:
motorcycles.sample(5)

,price,model,mileage,motorcycle_class,year,engine_capacity,engine_strokes,damaged,documents,city,date
id,,,,,,,,,,,
3031,460 000₽,Yamaha XVZ 1300,28 010 км,None,2004,1 300 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,12 августа 2016
1873,1 490 000₽,Harley-Davidson,393 км,Кастом,2007,2 000 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,5 мая 2017
1808,382 000₽,Harley-Davidson Sportster 1200 Custom XL1200C,20 610 км,Круизер,2007,1 200 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,7 мая 2017
6219,235 000₽,Yamaha Maxam 250,7 789 км,None,2009,250 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,28 июля 2013
5573,140 000₽,Yamaha XJ 900,60 000 км,None,1999,892 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,4 мая 2014


In [4]:
motorcycles.shape

(6848, 11)

Видим, что данные успешно загрузились и выглядят вполне адекватно. Можем работать дальше.

<br>

### 2. Первичный анализ данных

Посмотрим общую информацию по датасету:

In [5]:
motorcycles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6848 entries, 1 to 7100
Data columns (total 11 columns):
price               6848 non-null object
model               6848 non-null object
mileage             6848 non-null object
motorcycle_class    6848 non-null object
year                6848 non-null object
engine_capacity     6848 non-null object
engine_strokes      6848 non-null object
damaged             6848 non-null object
documents           6848 non-null object
city                6848 non-null object
date                6848 non-null object
dtypes: object(11)
memory usage: 642.0+ KB


In [6]:
motorcycles.nunique()

price               1059
model               2284
mileage             1958
motorcycle_class      15
year                  56
engine_capacity      348
engine_strokes         3
damaged                3
documents              3
city                   1
date                2133
dtype: int64

In [7]:
motorcycles.drop_duplicates().shape

(6797, 11)

Теперь посмотрим на некоторые столбцы, вызывающие вопросы:

**price:**

In [16]:
motorcycles.price.value_counts()

150 000₽           135
200 000₽           132
250 000₽           132
300 000₽           115
350 000₽           109
220 000₽           103
120 000₽           101
180 000₽            92
100 000₽            89
160 000₽            86
190 000₽            86
110 000₽            85
Цена не указана     85
450 000₽            83
230 000₽            83
170 000₽            83
130 000₽            80
280 000₽            74
140 000₽            72
165 000₽            71
240 000₽            71
400 000₽            69
90 000₽             63
260 000₽            60
210 000₽            58
125 000₽            58
320 000₽            58
270 000₽            58
500 000₽            55
290 000₽            54
                  ... 
39 990₽              1
290₽                 1
1 355 000₽           1
888 888₽             1
219 800₽             1
464 000₽             1
1 463 000₽           1
520₽                 1
558 000₽             1
111 500₽             1
1 357 000₽           1
474 000₽             1
21 900₽    

**model:**

In [29]:
motorcycles.model.value_counts()[:8]

Yamaha YZF R6             95
Honda CBR 600RR           73
Yamaha YZF R1             58
Suzuki Boulevard M109R    58
Yamaha FZ 6               55
Yamaha                    52
Kawasaki Ninja ZX-6R      51
Honda CB 400SF            51
Name: model, dtype: int64

**mileage:**

In [35]:
motorcycles.mileage.value_counts()[:10] / motorcycles.mileage.size

None         0.088201
1 км         0.068633
30 000 км    0.017377
20 000 км    0.016063
25 000 км    0.014895
40 000 км    0.010952
12 000 км    0.010806
15 000 км    0.009930
23 000 км    0.009784
10 000 км    0.009784
Name: mileage, dtype: float64

**motorcycle_class:**

In [13]:
motorcycles.motorcycle_class.value_counts() / motorcycles.motorcycle_class.size

None               0.518400
Чоппер             0.178592
Классика           0.067465
Спортивный         0.053154
Круизер            0.040888
Спорт-турист       0.037967
Эндуро             0.025263
Питбайк            0.024971
Туристический      0.018692
Стритфайтер        0.011536
Кроссовый          0.008470
Кастом             0.007155
Мотард             0.003213
Детский            0.002482
Трайк (трицикл)    0.001752
Name: motorcycle_class, dtype: float64

**year:**

In [25]:
motorcycles.year.value_counts()[:10] / motorcycles.year.size

2014    0.070093
2013    0.063960
2008    0.062354
2006    0.059725
2007    0.058557
2005    0.047605
2012    0.045853
2009    0.044539
2003    0.043954
None    0.043808
Name: year, dtype: float64

**engine_capacity:**

In [28]:
motorcycles.engine_capacity.value_counts()[:13] / motorcycles.engine_capacity.size

400 куб. см.      0.110981
600 куб. см.      0.097985
250 куб. см.      0.088347
1 000 куб. см.    0.049504
650 куб. см.      0.045999
750 куб. см.      0.036069
1 200 куб. см.    0.035193
1 300 куб. см.    0.032856
125 куб. см.      0.027599
1 800 куб. см.    0.025847
1 100 куб. см.    0.025701
800 куб. см.      0.024533
None              0.019860
Name: engine_capacity, dtype: float64

**engine_strokes:**

In [10]:
motorcycles.engine_strokes.value_counts() / motorcycles.engine_strokes.size

4х тактный    0.863902
None          0.081484
2х тактный    0.054614
Name: engine_strokes, dtype: float64

**damaged:**

In [11]:
motorcycles.damaged.value_counts() / motorcycles.damaged.size

Исправен      0.968166
None          0.019568
Неисправен    0.012266
Name: damaged, dtype: float64

**documents:**

In [37]:
motorcycles.documents.value_counts() / motorcycles.documents.size

Есть ПТС    0.908732
Без ПТС     0.089223
None        0.002044
Name: documents, dtype: float64

**date:**

In [36]:
motorcycles.sample(10).date

id
3139        15 июля 2016
180           1 сентября
2093      27 апреля 2017
5125      26 ноября 2014
266            2 августа
21             19 ноября
4525        28 июля 2015
5987       1 ноября 2013
1099    11 сентября 2017
3383         30 мая 2016
Name: date, dtype: object

Теперь пройдемся по столбцам и, на основе уже имеющейся информации, дадим им некоторое описание:
1. **id:** просто порядковый номер мотоцикла, который вряд ли несет какую либо полезную информацию.
2. **price:** цена мотоцикла, целевая переменная. Видим, что есть объекты со значением "Цена не указана". Их нам придется отбросить. Также можно заметить, что не у всех мотоциклов стоит адекватная цена (290₽ и т.п.). Такие объекты тоже скорее всего можно отбросить, так как они вряд ли составляют весомую долю от всех остальных. Также следует преобразовать стоимость в числовой тип, отбросив символ "₽".
3. **model:** модель мотоцикла. Можно построить отдельный столбец **manufacturer**, в котором указывать только производителя, и рассматривать его как новое свойство.
4. **mileage:** пробег мотоцикла. 9% имею пропуски, возможно это из за того, что такие мотоциклы являются новыми. Было бы странно, если бы продавец не указывал хотя бы примерный пробег своего подержанного транспортного средства, ведь это является один из самых главных факторов при покупке. Так что заполним пропуски нулями. Также 7% имеют пробег "1 км", это странное значение можно тоже занулить. Значения стоит привести к числовым, отбросить подстроку "км".
5. **motorcycle_class:** класс мотоцикла. Как видим, около 50% всех объектов не имеют информции о классе. Возможно, этот столбец придется убрать из рассмотрения, так как заполнить пропуски сложно без искажения реальной информации. Считать отсутстиве значения как свойство не имеет смысла, так как класс присущ каждому мотоциклу, и отстуствие информации об этом характеризует скорее не мотоцикл, а человека, составляющего объявление.
6. **year:** год выпуска мотоцикла. Как видим, значения отсутсвуют лишь у 4% всех объявлений, что не значительно.
7. **engine_capacity:** объем двигателя. Пропуски имеют 2% объектов. Значения можно преобразовать с числовой тип, отбросив подстроку "куб. см.".
8. **engine_strokes:** число тактов двигателя. Видим, что всего 2 разных значения, и что больше 85% мотоциклов обладают четырехтактным двигателем. Также имеем 8% пропусков. Можно попробовать заменить пропуски на "4х тактный", так как большинство двигателей именно такие, но можно и вообще не рассматривать этот столбец. На данном этапе пока оставим, лишь обрежем строку до одного числа, 2 или 4.
9. **damaged:** состояние мотоцикла. Пропуски имеют 2%, и с большой уверенностью можно заполнить их значениями "Исправен", так как предполагается, что если мотоцикл сломан, то продавец об этом скорее всего сообщит заранее (если он добросовестный, конечно). Также значение "Исправен" можно закодировать под 0, а "Неисправен" - под 1.
10. **documents:** наличие ПТС у продавца. Аналогично с прошлым столбцом, можно заполнить пропуски самым популярным вариантом и закодировать "Есть ПТС" как 0 и "Без ПТС" как 1.
11. **city:** все объявления из одного города, не несет информации.
12. **date:** как можно заметить из семпла, у некоторых объявлений есть информация о годе, а у некоторых нет. Те, у которых нет - объявления за 2018 год. Надо подправить это и дописать 2018 к этим датам.

<br>

### 3. Предобработка данных